# --- `Day 16`: Title ---

In [1]:
import aocd
import re
import heapq
import operator
from collections import Counter, defaultdict, deque
from itertools import combinations
from functools import reduce, lru_cache

def prod(iterable):
    return reduce(operator.mul, iterable, 1)

def count(iterable, predicate = bool):
    return sum([1 for item in iterable if predicate(item)])

def first(iterable, default = None):
    return next(iter(iterable), default)

def lmap(func, *iterables):
    return list(map(func, *iterables))

def ints(s):
    return lmap(int, re.findall(r"-?\d+", s))

def words(s):
    return re.findall(r"[a-zA-Z]+", s)

def list_diff(x):
    return [b - a for a, b in zip(x, x[1:])]

def binary_to_int(lst):
    return int("".join(str(i) for i in lst), 2)

def get_column(lst, index):
    return [x[index] for x in lst]

In [2]:
def parse_line(line): 
    return str(line)
    
def parse_input(input):
    return list(map(parse_line, input.splitlines()))

In [3]:
final_input = parse_input(aocd.get_data(day=16, year=2021))
print(final_input[:5])

['20546718027401204FE775D747A5AD3C3CCEEB24CC01CA4DFF2593378D645708A56D5BD704CC0110C469BEF2A4929689D1006AF600AC942B0BA0C942B0BA24F9DA8023377E5AC7535084BC6A4020D4C73DB78F005A52BBEEA441255B42995A300AA59C27086618A686E71240005A8C73D4CF0AC40169C739584BE2E40157D0025533770940695FE982486C802DD9DC56F9F07580291C64AAAC402435802E00087C1E8250440010A8C705A3ACA112001AF251B2C9009A92D8EBA6006A0200F4228F50E80010D8A7052280003AD31D658A9231AA34E50FC8010694089F41000C6A73F4EDFB6C9CC3E97AF5C61A10095FE00B80021B13E3D41600042E13C6E8912D4176002BE6B060001F74AE72C7314CEAD3AB14D184DE62EB03880208893C008042C91D8F9801726CEE00BCBDDEE3F18045348F34293E09329B24568014DCADB2DD33AEF66273DA45300567ED827A00B8657B2E42FD3795ECB90BF4C1C0289D0695A6B07F30B93ACB35FBFA6C2A007A01898005CD2801A60058013968048EB010D6803DE000E1C6006B00B9CC028D8008DC401DD9006146005980168009E1801B37E02200C9B0012A998BACB2EC8E3D0FC8262C1009D00008644F8510F0401B825182380803506A12421200CB677011E00AC8C6DA2E918DB454401976802F29AA324A6A8C12B3FD978004EB30076194278BE600

In [170]:
test_input = parse_input('''\
9C0141080250320F1802104A08
''')

print(test_input)

['9C0141080250320F1802104A08']


### Helpers

In [52]:
print(bin(int('3', 16))[2:].zfill(4))

bits = [x for x in bin(int('3', 16))[2:].zfill(4)]
print(bits)

0011
['0', '0', '1', '1']


## Solution 1

In [160]:
def readLiteral(rest):
    w = len(rest)
    result = []
    while True:
        A, rest = rest[:5], rest[5:]
        if A[0] == '1':
            result.extend(A[1:])
        else:
            result.extend(A[1:])
            break
    literal = binary_to_int(result)
    return literal, rest

def readOperator(rest):
    count = 0
    lengthType, rest = rest[0], rest[1:]
    if lengthType == '0':
        totalLength, rest = rest[:15], rest[15:]
        totalLength = binary_to_int(totalLength)
        
        total = 0
        while True:
            start = len(rest)
            rest, version = parse(rest)
            count += version
            read = start - len(rest)
            total += read
            if total == totalLength:
                break
    else:
        numberSub, rest = rest[:11], rest[11:]
        numberSub = binary_to_int(numberSub)
        
        for _ in range(numberSub):
            rest, version = parse(rest)
            count += version
    return rest, count

def readHeader(binary):
    version, rest = binary[:3], binary[3:]
    version = binary_to_int(version)
    
    packetId, rest = rest[:3], rest[3:]
    packetId = binary_to_int(packetId)
    return version, packetId, rest

def parse(bits):
    version, packetId, rest = readHeader(bits)
    print("version", version, "type", packetId)

    count = 0
    if packetId == 4:
        literal, rest = readLiteral(rest)
    else:
        rest, count = readOperator(rest)
    return rest, version + count
    
def solve_1(input):
    binary = []
    for c in input[0]:
        bits = [x for x in bin(int(c, 16))[2:].zfill(4)]
        binary.extend(bits)
        
    rest, count = parse(binary)
    return count

solve_1(test_input)

version 6 type 6
version 5 type 4
version 2 type 4


13

In [ ]:
f"Solution 1: {solve_1(final_input)}"

## Solution 2

In [171]:
def readLiteral2(rest):
    w = len(rest)
    result = []
    while True:
        A, rest = rest[:5], rest[5:]
        if A[0] == '1':
            result.extend(A[1:])
        else:
            result.extend(A[1:])
            break
    literal = binary_to_int(result)
    print("literal", literal)
    return literal, rest

def readOperator2(rest, packetId):
    count = 0
    values = []
    lengthType, rest = rest[0], rest[1:]
    if lengthType == '0':
        totalLength, rest = rest[:15], rest[15:]
        totalLength = binary_to_int(totalLength)
        
        total = 0
        while True:
            start = len(rest)
            rest, value = parse2(rest)
            values.append(value)
            read = start - len(rest)
            total += read
            if total == totalLength:
                break
    else:
        numberSub, rest = rest[:11], rest[11:]
        numberSub = binary_to_int(numberSub)
        
        for _ in range(numberSub):
            rest, value = parse2(rest)
            values.append(value)
            
    if packetId == 0:
        return rest, sum(values)
    elif packetId == 1:
        return rest, prod(values)
    elif packetId == 2:
        return rest, min(values)
    elif packetId == 3:
        return rest, max(values)
    elif packetId == 5:
        value = 1 if values[0] > values[1] else 0
        return rest, value
    elif packetId == 6:
        value = 1 if values[0] < values[1] else 0
        return rest, value
    elif packetId == 7:
        value = 1 if values[0] == values[1] else 0
        return rest, value
    return rest, value

def readHeader2(binary):
    version, rest = binary[:3], binary[3:]
    version = binary_to_int(version)
    
    packetId, rest = rest[:3], rest[3:]
    packetId = binary_to_int(packetId)
    return version, packetId, rest

def parse2(bits):
    version, packetId, rest = readHeader2(bits)
    print("version", version, "type", packetId)

    value = 0
    if packetId == 4:
        value, rest = readLiteral2(rest)
    else:
        rest, value = readOperator2(rest, packetId)
    return rest, value
    
def solve_2(input):
    binary = []
    for c in input[0]:
        bits = [x for x in bin(int(c, 16))[2:].zfill(4)]
        binary.extend(bits)
        
    rest, value = parse2(binary)
    return value
    
solve_2(test_input)

version 4 type 7
version 2 type 0
version 2 type 4
literal 1
version 4 type 4
literal 3
version 6 type 1
version 0 type 4
literal 2
version 2 type 4
literal 2


1

In [172]:
f"Solution 2: {solve_2(final_input)}"

version 1 type 0
version 6 type 1
version 3 type 5
version 0 type 4
literal 261839571
version 6 type 4
literal 418703
version 1 type 4
literal 55956
version 6 type 3
version 4 type 4
literal 49045
version 4 type 4
literal 9901349
version 3 type 4
literal 4
version 2 type 4
literal 90535300
version 6 type 3
version 1 type 4
literal 8
version 6 type 4
literal 777300
version 4 type 4
literal 17943
version 2 type 1
version 7 type 5
version 5 type 4
literal 2117752
version 1 type 4
literal 2117752
version 4 type 4
literal 247
version 3 type 2
version 1 type 4
literal 12043879
version 2 type 4
literal 10486502
version 5 type 1
version 2 type 4
literal 577215
version 0 type 7
version 6 type 4
literal 73128484
version 0 type 4
literal 39191707
version 2 type 1
version 2 type 4
literal 110
version 0 type 4
literal 196
version 1 type 4
literal 140
version 2 type 4
literal 163
version 3 type 4
literal 201
version 1 type 0
version 2 type 4
literal 12
version 3 type 4
literal 936389
version 3 type 

'Solution 2: 158135423448'